In [6]:
import datetime
from calculator import LoanContext
from calculator.rates import InterestContext, InflationContext
from calculator.loans.downpayments import DownPayments
from calculator.loans.indexedloan import IndexBasedLoan

lcontext = LoanContext(40000, datetime.datetime.now(), 25)
icontext = InterestContext(2.6)
fcontext = InflationContext(9.57)
dpayments = DownPayments(lcontext, icontext)
a = IndexBasedLoan(dpayments, fcontext)
a.calculate()
a.df

,date,capital_base,payments_base,interests_base,instalments_base,inflation_csum,capital,payment,interest,instalments
0,2023-03-01 00:15:18.199278,40000.000000,181.467803,86.666667,94.801136,0.007975,40319.000000,182.915009,87.357833,95.557175
1,2023-04-01 00:15:18.199278,39905.198864,181.467803,86.461264,95.006539,0.015950,40541.686786,184.362214,87.840321,96.521893
2,2023-05-01 00:15:18.199278,39810.192325,181.467803,86.255417,95.212386,0.023925,40762.651176,185.809420,88.319078,97.490343
3,2023-06-01 00:15:18.199278,39714.979939,181.467803,86.049123,95.418680,0.031900,40981.887799,187.256626,88.794090,98.462536
4,2023-07-01 00:15:18.199278,39619.561259,181.467803,85.842383,95.625420,0.039875,41199.391264,188.703832,89.265348,99.438484
...,...,...,...,...,...,...,...,...,...,...
295,2047-10-01 00:15:18.199278,901.470998,181.467803,1.953187,179.514616,2.360600,3029.483437,609.840698,6.563881,603.276818
296,2047-11-01 00:15:18.199278,721.956383,181.467803,1.564239,179.903564,2.368575,2431.964221,611.287904,5.269256,606.018648
297,2047-12-01 00:15:18.199278,542.052819,181.467803,1.174448,180.293355,2.376550,1830.268444,612.735110,3.965582,608.769528
298,2048-01-01 00:15:18.199278,361.759463,181.467803,0.783812,180.683991,2.384525,1224.383948,614.182316,2.652832,611.529484


In [5]:
a.df.loc[0].capital_base * (1 + 0.54/100)

45243.0